In [136]:
import pandas as pd
ff="model_ans_llama_finetuned_rag_486"
df=pd.read_excel(r"excel file\new\llama\model_ans_llama_finetuned_without_rag_486.xlsx")
global cnt
cnt=0
df.head()

,id,question,answer
0,1,"How many students can study in CSE,Civil, EEE ...",Hello! I'm happy to help you with your query. ...
1,2,What does KUET stands for?,Hello! I'm here to help you with any questions...
2,3,Where is the Academic building ar located in t...,\nHello! I'm happy to help you with your query...
3,4,How many hall are allocated for Boys in KUET?,Hello! I'm happy to help you with your query a...
4,5,Where is KUET located?,\nHello! I'm happy to help you with your query...


In [137]:
data=[]
import datetime
current_datetime = datetime.datetime.now()
file_name = current_datetime.strftime("%Y_%m_%d_%H_%M_%S")+ff
file_name

'2024_03_21_23_39_45model_ans_llama_finetuned_rag_486'

In [138]:
def random_ques_ans():
    import random
    # id=random.randint(0,len(df)-1)
    global cnt
    id=int((df.loc[cnt])['id'])
    ques_temp=(df.loc[cnt])['question']
    ans_temp=(df.loc[cnt])['answer']
    cnt+=1
    if cnt>=len(df):
        cnt=0
        return ques_temp,ans_temp,id,0

    return ques_temp,ans_temp,id,1

In [139]:
# def correct_cnt(ques,ans):
#     data.append({
#         "question":ques,
#         'answer':ans,
#         'correct_wrong':'correct'
#     })
#     if len(data)%10==0:
#         temp=pd.DataFrame(data)
#         temp.to_excel(f"data//{file_name}.xlsx",index=False)
#     ques_temp,ans_temp=random_ques_ans()
#     gr.Info("Your opinion is submitted successfully!!!")
#     return gr.Label(value=ques_temp, label="Question"), gr.Label(value=ans_temp, label="Answer")
    
# def wrong_cnt(ques,ans):
#     data.append({
#         "question":ques,
#         'answer':ans,
#         'correct_wrong':'wrong'
#     })
#     if len(data)%10==0:
#         temp=pd.DataFrame(data)
#         temp.to_excel(f"data//{file_name}.xlsx",index=False)
#     ques_temp,ans_temp=random_ques_ans()
#     gr.Info("Your opinion is submitted successfully!!!")
#     return gr.Label(value=ques_temp, label="Question"), gr.Label(value=ans_temp, label="Answer")
# def new_ques():
#     ques_temp,ans_temp=random_ques_ans()
#     return {
#         ques:gr.Label(value=ques_temp,label="Question"),
#         ans:gr.Label(value=ans_temp,label="Answer")
#     }

In [140]:
def save_all():
    temp=pd.DataFrame(data)
    temp.to_excel(f"score_report//{file_name}.xlsx",index=False)
    gr.Info("Sucessfully save all the answer!!!")
def score_save(ques,ans,score):
    data.append({
        "question":ques,
        'answer':ans,
        'rating':score
    })
    print('*'*10,":",data[len(data)-1])
    if len(data)%5==0:
        temp=pd.DataFrame(data)
        temp.to_excel(f"score_report//{file_name}.xlsx",index=False)
    ques_temp,ans_temp,id,flag=random_ques_ans()
    if flag==0:
        temp=pd.DataFrame(data)
        temp.to_excel(f"score_report//{file_name}.xlsx",index=False)
        gr.Info("Thank for your contribution. Revision is completed for now!!!")
        assert(False)
    gr.Info("Your opinion is submitted successfully!!!")
    return gr.Label(value=id,label="ID"),gr.Label(value=ques_temp, label="Question"), gr.Label(value=ans_temp, label="Answer")
def new_ques():
    ques_temp,ans_temp,id,flag=random_ques_ans()
    if flag==0:
        temp=pd.DataFrame(data)
        temp.to_excel(f"score_report//{file_name}.xlsx",index=False)
        gr.Info("Thank for your contribution. Revision is completed for now!!!")
        assert(False)
    return {
        id:gr.Label(value=id,label="ID"),
        ques:gr.Label(value=ques_temp,label="Question"),
        ans:gr.Label(value=ans_temp,label="Answer")
    }

In [141]:
def move_to(move):
    if int(move)>=len(df)+1:
        gr.Info(f"Number . of questions: {len(df)}")
        move=0
    id_temp=(df.loc[move-1])['id']   
    ques_temp=(df.loc[move-1])['question']
    ans_temp=(df.loc[move-1])['answer']
    return {
        id:gr.Label(value=str(id_temp),label="ID"),
        ques:gr.Label(value=ques_temp,label="Question"),
        ans:gr.Label(value=ans_temp,label="Answer")
    }

In [142]:
import gradio as gr
import pandas as pd
css = """
#accepted {background-color: green;align-content: center;font: 30px Arial, sans-serif;}
#wrong {background-color: red;align-content: center;font: 30px Arial, sans-serif;}
#already {background-color: blue;align-content: center;font: 30px Arial, sans-serif;}
"""
ques_temp,ans_temp,id_temp,flag=random_ques_ans()
with gr.Blocks(css=css) as demo:
    gr.Markdown(
    f"""
    # Model name: {ff}
    # No. of questions:{len(df)} 
    
    """)
    with gr.Row():
        id=gr.Label(value=id_temp,label="ID")
        avg_rating=gr.Label(visible=False)
    with gr.Row():
        ques=gr.Label(value=ques_temp,label="Question")
    with gr.Row():
        ans=gr.Label(value=ans_temp,label="Answer")
    with gr.Row():
        # correct = gr.Button("Correct")
        # wrong = gr.Button("Wrong")
        score = gr.Radio(choices=[1,2,3,4,5],label="Rating")
    with gr.Row():
        btn = gr.Button("Save")
        question = gr.Button("Generate new question")
    with gr.Row():
        save_all_btn=gr.Button("Save all the data in dataframe")
    with gr.Row():
        move=gr.Number(label="Move to the question")
        move_btn=gr.Button("move")
    # correct.click(correct_cnt,[ques,ans],[ques,ans])
    # wrong.click(wrong_cnt,[ques,ans],[ques,ans])
    
    btn.click(score_save, inputs=[ques,ans,score], outputs=[ques,ans])
    question.click(new_ques,None,[ques,ans])
    save_all_btn.click(save_all,None,None)
    move_btn.click(move_to,move,[id,ques,ans])

demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7880
Running on public URL: https://caf98d8c6e97171c2a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
